In [ ]:
import logging
import pathlib

import plotly.express as px
import pandas as pd
import numpy as np


import lns
from lns import cvrp
from lns import get_logger


logger = get_logger("main", level=logging.WARNING)

In [ ]:
# this notebook assumes that benchmarking
# was already run using benchmark.py script
# (actually, it expects 2 separate runs of that script, with baseline and tuned parameters)
experiments = pd.read_csv("../data/benchmark.csv")
baseline_experiments = pd.read_csv("../data/baseline-benchmark.csv")
experiments["mape-best-percent"] = experiments["mape-best"].map(lambda x: x * 100)
baseline_experiments["mape-best-percent"] = baseline_experiments["mape-best"].map(
    lambda x: x * 100
)
experiments["mape-initial-percent"] = experiments["mape-initial"].map(lambda x: x * 100)
baseline_experiments["mape-initial-percent"] = baseline_experiments["mape-initial"].map(
    lambda x: x * 100
)

In [ ]:
experiments.sort_values(by="mape-best", ascending=False)["name"]

In [ ]:
experiments["exp"] = "tuned"
baseline_experiments["exp"] = "baseline"
df = pd.concat((experiments, baseline_experiments))
df = df[~df["name"].isin(("E-n31-k7", "E-n13-k4"))]
df.groupby(by=["exp", "subset"], as_index=False).aggregate(
    {"mape-best-percent": "mean"}
)
df["mape-best-percent"] = df["mape-best-percent"].map(lambda x: np.round(x, 3))

In [ ]:
def plot_set_mape(df: pd.DataFrame, title: str = ""):
    df = df[df["mape-best"] < 0.5]
    df = df.groupby(by=["exp", "subset"], as_index=False).aggregate(
        {"mape-best-percent": "mean"}
    )
    df["mape-best-percent-round"] = df["mape-best-percent"].map(lambda x: f"{x:.3f}%")
    fig = px.bar(
        df,
        x="subset",
        color="exp",
        y="mape-best-percent",
        barmode="group",
        text="mape-best-percent-round",
        title=title,
    )

    # fig.update_traces(textposition="outside", textfont_size=12)
    return fig

In [ ]:
plot_set_mape(df, "Mean MAPE by subset, 2 outliers from set E removed (%)").show()

In [ ]:
def plot_mape_scores(df: pd.DataFrame, title_prefix: str = ""):
    fig = px.scatter(
        df,
        x="dimension",
        y=["mape-best-percent"],
        color="subset",
        hover_data="name",
        title=title_prefix + " best MAPE (%), lower is better",
    )

    fig.add_hline(y=0.0, line_width=0.7, line_dash=None, line_color="black")

    return fig


def plot_iterations(df: pd.DataFrame, title_suffix: str = ""):
    fig = px.scatter(
        df,
        x="dimension",
        y="time",
        color="subset",
        hover_data="name",
        title="Execution time (s) / problem size" + title_suffix,
    )

    return fig


def plot_initial_solution(df: pd.DataFrame):
    fig = px.scatter(
        df,
        x="dimension",
        y="mape-initial-percent",
        color="subset",
        hover_data="name",
        title="MAPE (%) of initial solution / problem size",
    )

    fig.add_hline(y=0.0, line_width=0.7, line_dash=None, line_color="black")
    return fig

In [ ]:
plot_mape_scores(
    baseline_experiments[~baseline_experiments["name"].isin(("E-n31-k7", "E-n13-k4"))],
    "ALNS (baseline, random remove + greedy repair + random accept)",
).show()
plot_mape_scores(
    experiments[~experiments["name"].isin(("E-n31-k7", "E-n13-k4"))],
    "ALNS (random remove / SISR + greedy repair + SA accept)",
).show()

In [ ]:
plot_iterations(baseline_experiments, " (baseline)").show()
plot_iterations(experiments, " (after hyperparameter tuning)").show()

In [ ]:
plot_initial_solution(experiments).show()

In [ ]:
from itertools import pairwise


def plot_solution_routes(p: cvrp.Problem, sol: cvrp.Solution, title: str):
    df = pd.DataFrame(
        {
            "x": p.customers[:, 0],
            "y": p.customers[:, 1],
            "customer": range(len(p.customers)),
            "marker": "customer",
            "demand": p.demands,
            "marker_size": 0.2,
        }
    )

    df.loc[0, ["marker", "marker_size"]] = "depot", 3

    fig = px.scatter(
        df,
        x="x",
        y="y",
        symbol="marker",
        symbol_sequence=["star", "circle-open"],
        size="marker_size",
        hover_data="customer",
        title=title,
        height=800,
        width=1200,
    )

    for i, route in enumerate(sol.routes, start=1):
        edge_x, edge_y = [], []

        for a, b in pairwise([0] + route + [0]):
            edge_x.append(p.customers[a, 0])
            edge_x.append(p.customers[b, 0])
            edge_x.append(None)

            edge_y.append(p.customers[a, 1])
            edge_y.append(p.customers[b, 1])
            edge_y.append(None)

        fig.add_scatter(
            x=edge_x,
            y=edge_y,
            name=f"Route {i}",
            showlegend=False,
        )

    return fig

In [ ]:
from benchmark import benchmark_model, read_problem


def better_alns_factory(
    seed: int = 10, max_iterations: int = 10_000, max_runtime: float = 60
):
    # ensure reentrance
    rng = np.random.default_rng(seed=seed)

    def solve(p: cvrp.Problem, initial: cvrp.Solution):
        accept_criterion = lns.accept.SimulatedAnnealing.fit(
            initial.cost,
            worse=0.5,
            accept_proba=0.1,
            num_iters=max_iterations,
            method="exponential",
        )

        cfg = lns.operators.BasicDestroyConfig(
            problem=p,
            bounds=[min(5, 0.1 * p.dim), min(50, 0.5 * p.dim)],
            rng=rng,
        )

        destroy_operators = [
            lns.operators.RandomRemove(cfg),
            lns.operators.SubstringRemoval(
                max_substring_removals=2,
                max_string_size=12,
                cfg=cfg,
            ),
        ]

        repair_operators = [
            lns.operators.GreedyRepair(
                lns.operators.BasicRepairConfig(
                    problem=p,
                    rng=rng,
                )
            )
        ]

        solver = lns.alns.ALNS(
            accept=accept_criterion,
            destroy_operators=destroy_operators,
            repair_operators=repair_operators,
        )

        return solver.iterate(
            initial,
            max_iter=max_iterations,
            max_runtime=max_runtime,
            verbose=True,
            handle_interrupts=False,
        )

    return solve

In [ ]:
problems_to_inspect = experiments.sort_values(by="mape-best", ascending=False)[
    ["name", "mape-best"]
][:5]
problems_to_inspect

In [ ]:
def plot_iterations(solution: lns.alns.TracedSolution, optimal_cost: float, title: str):
    df = pd.DataFrame(
        {
            "best": solution.best_costs,
            "running": solution.iteration_costs,
        }
    )

    mape = (solution.best_solution.cost - optimal_cost) / optimal_cost

    fig = px.scatter(
        df,
        y="running",
        height=800,
        width=1200,
        title=f"{title}: Solution progress: best MAPE: {mape * 100:.3f}%",
    )

    fig.add_scatter(
        y=df.best,
        mode="lines",
        name="best",
    )

    fig.add_hline(y=optimal_cost)
    fig.update_traces(marker_size=2.5)
    return fig

In [ ]:
benchmarking_reports = [
    benchmark_model(
        name,
        p,
        opt,
        model=better_alns_factory(max_iterations=100_000, max_runtime=300),
        builder=lns.construct.random_builder,
    )
    for name, p, opt in map(
        lambda x: read_problem(x, root=pathlib.Path("../data/")),
        problems_to_inspect["name"][2:],
    )
]

In [ ]:
_, _, case, *_ = benchmarking_reports
plot_solution_routes(
    case.problem,
    case.initial_solution,
    title=f"{case.name} initial solution (greedy NN with FPS seeding), MAPE {case.mape_init * 100:.3f}%",
).show()
plot_solution_routes(
    case.problem,
    case.alns_solution.best_solution,
    title=f"{case.name} best ALNS solution: {case.alns_solution.best_solution.cost:.3f}"
    f" ({case.alns_solution.iterations} iterations in {case.alns_solution.elapsed_time:3.2f}s, MAPE {case.mape * 100:.3f}%)",
).show()
plot_solution_routes(
    case.problem,
    case.opt_solution,
    title=f"{case.problem.name} (optimal solution - {case.opt_solution.cost})",
).show()
plot_iterations(
    case.alns_solution,
    case.opt_solution.cost,
    title=case.name,
).show()

In [ ]:
# looks like this instance did not had enough time to be solved
name = "M-n121-k7"
_, p, opt = read_problem(name, root=pathlib.Path("../data/"))
report = benchmark_model(
    name,
    p,
    opt,
    model=better_alns_factory(max_iterations=100_000, max_runtime=300),
    builder=lns.construct.random_builder,
)

In [ ]:
plot_solution_routes(
    report.problem,
    report.opt_solution,
    title=f"{report.problem.name} optimal solution",
).show()
plot_solution_routes(
    report.problem,
    report.initial_solution,
    title=f"{report.problem.name} initial solution (random), MAPE {report.mape_init * 100:.3f}%",
).show()
plot_solution_routes(
    report.problem,
    report.alns_solution.best_solution,
    title=f"{report.name} best ALNS solution: {report.alns_solution.best_solution.cost:.3f}"
    f" ({report.alns_solution.iterations} iterations in {report.alns_solution.elapsed_time:3.2f}s, MAPE {report.mape * 100:.3f}%)",
).show()
plot_iterations(
    report.alns_solution,
    report.opt_solution.cost,
    title=report.name,
).show()